In [21]:
#initialising the libraries

from ultralytics import YOLO
import os

In [22]:
#defining all the paths

# Main directories
BASE_DIR = "/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges"
DATASET_DIR = os.path.join(BASE_DIR, "Dataset")

# Dataset YAML path
DATASET_YAML = DATASET_YAML = os.path.join(BASE_DIR, "dataset.yaml")


# Pre-trained YOLO model weights
MODEL_PATH = os.path.join(BASE_DIR, "Core Model/yolov8s.pt")




In [23]:
# Create dataset.yaml
yaml_content = f"""
path: {DATASET_DIR}
train: train/images
val: valid/images

nc: 2  # Number of classes
names: [plastics, river]  # Class names
"""

# Save the YAML file
with open(DATASET_YAML, "w") as f:
    f.write(yaml_content)
print("Dataset YAML created successfully!")


Dataset YAML created successfully!


In [24]:
# Initialize the YOLO model with pre-trained weights
model = YOLO(MODEL_PATH)

/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Core Model/yolov8s.pt

In [34]:
#converting yolo to labels


import os
import json

# Paths
dataset_dir = "/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset"
splits = ["train", "valid", "test"]  # Update based on your dataset

for split in splits:
    json_path = os.path.join(dataset_dir, split, "_annotations.coco.json")
    images_dir = os.path.join(dataset_dir, split, "images")
    labels_dir = os.path.join(dataset_dir, split, "labels")

    # Create labels directory if not exists
    os.makedirs(labels_dir, exist_ok=True)

    # Load COCO JSON
    with open(json_path, "r") as f:
        coco_data = json.load(f)

    # Map image IDs to file names
    image_id_to_filename = {img["id"]: img["file_name"] for img in coco_data["images"]}

    # Process annotations
    for ann in coco_data["annotations"]:
        image_id = ann["image_id"]
        class_id = ann["category_id"] - 1  # Class IDs should start from 0
        bbox = ann["bbox"]

        # Convert COCO bbox [x, y, width, height] to YOLO format
        x, y, width, height = bbox
        image_width, image_height = next(
            (img["width"], img["height"]) for img in coco_data["images"] if img["id"] == image_id
        )
        x_center = (x + width / 2) / image_width
        y_center = (y + height / 2) / image_height
        width /= image_width
        height /= image_height

        # Write to corresponding label file
        label_file = os.path.join(labels_dir, f"{image_id_to_filename[image_id].split('.')[0]}.txt")
        with open(label_file, "a") as f:
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

print("Conversion completed! YOLO labels are ready.")


Conversion completed! YOLO labels are ready.


In [25]:
import json
import os

# Paths
dataset_dir = "/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset/train"
coco_path = os.path.join(dataset_dir, "_annotations.coco.json")

# Load JSON
with open(coco_path, "r") as f:
    data = json.load(f)

# Check images
missing_images = []
for img in data["images"]:
    img_path = os.path.join(dataset_dir, "images", img["file_name"])
    if not os.path.exists(img_path):
        missing_images.append(img["file_name"])

if missing_images:
    print("Missing images:", missing_images)
else:
    print("All images found!")


All images found!


In [26]:

# Train the model
model.train(
    data="/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset.yaml",  # Path to YAML
    epochs=50,      # Number of epochs (adjust as needed)
    imgsz=640,      # Image size
    batch=16,       # Batch size
    workers=4,      # Number of data-loading workers
    name="river_plastic_detection"  # Experiment name
)


Ultralytics 8.3.54 🚀 Python-3.12.0 torch-2.5.1 CPU (Apple M1)
engine/trainer: task=detect, mode=train, model=/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Core Model/yolov8s.pt, data=/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=4, project=None, name=river_plastic_detection, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fal

train: Scanning /Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset/train/labels.cache... 0 images, 1026 backgrounds, 0 corrupt: 100%|██████████| 1026/1026 [00:00<?, ?it/s]

WARNING ⚠️ No labels found in /Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset/train/labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.



val: Scanning /Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset/valid/labels.cache... 0 images, 43 backgrounds, 0 corrupt: 100%|██████████| 43/43 [00:00<?, ?it/s]

WARNING ⚠️ No labels found in /Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset/valid/labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
Plotting labels to runs/detect/river_plastic_detection/labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)


...

       1/50         0G          0      61.64          0          0        640: 100%|██████████| 65/65 [19:04<00:00, 17.60s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:09<00:00,  4.56s/it]

                   all         43          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       2/50         0G          0      2.909          0          0        640:  25%|██▍       | 16/65 [05:25<16:37, 20.35s/it]


KeyboardInterrupt: 

/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset/train/labels

In [12]:
import os

train_labels = "/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset/train/labels"
valid_labels = "/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset/valid/labels"

print("Train Labels: ", os.listdir(train_labels))
print("Valid Labels: ", os.listdir(valid_labels))


Train Labels:  ['Aerial_Location_1_194_jpg.txt', 'Aerial_Location_1_184_jpg.txt', 'Aerial_Location_1_81_jpg.txt', 'Aerial_Location_1_91_jpg.txt', 'Aerial_Location_1_333_jpg.txt', 'Aerial_Location_1_323_jpg.txt', 'Aerial_Location_1_38_jpg.txt', 'Aerial_Location_1_229_jpg.txt', 'Aerial_Location_1_351_jpg.txt', 'Aerial_Location_1_341_jpg.txt', 'Aerial_Location_1_290_jpg.txt', 'Aerial_Location_1_280_jpg.txt', 'Aerial_Location_1_112_jpg.txt', 'Aerial_Location_1_274_jpg.txt', 'Aerial_Location_1_264_jpg.txt', 'Aerial_Location_1_421_jpg.txt', 'Aerial_Location_1_65_jpg.txt', 'Aerial_Location_1_75_jpg.txt', 'Aerial_Location_1_216_jpg.txt', 'Aerial_Location_1_206_jpg.txt', 'Aerial_Location_1_170_jpg.txt', 'Aerial_Location_1_160_jpg.txt', 'Aerial_Location_1_156_jpg.txt', 'Aerial_Location_1_146_jpg.txt', 'Aerial_Location_1_43_jpg.txt', 'Aerial_Location_1_53_jpg.txt', 'Aerial_Location_1_230_jpg.txt', 'Aerial_Location_1_348_jpg.txt', 'Aerial_Location_1_220_jpg.txt', 'Aerial_Location_1_289_jpg.txt', '

In [15]:
len(train_labels)

93

In [35]:
import os

dataset_dir = "/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset"
splits = ["train", "valid"]

for split in splits:
    images_path = os.path.join(dataset_dir, split, "images")
    labels_path = os.path.join(dataset_dir, split, "labels")

    images = sorted(os.listdir(images_path))
    labels = sorted(os.listdir(labels_path))

    print(f"{split} - Images: {len(images)}, Labels: {len(labels)}")

    for img in images:
        img_name = os.path.splitext(img)[0]
        label_file = os.path.join(labels_path, f"{img_name}.txt")
        if not os.path.exists(label_file):
            print(f"Missing label for: {img}")


train - Images: 499, Labels: 342
Missing label for: .DS_Store
Missing label for: Aerial_Location_1_100_jpg.rf.5be0977b279a691b999e19511352f30e.jpg
Missing label for: Aerial_Location_1_100_jpg.rf.659b25171e747f1c7fc22a29885d4e31.jpg
Missing label for: Aerial_Location_1_101_jpg.rf.1564c0f4fda15d157121bab7f51e1de8.jpg
Missing label for: Aerial_Location_1_101_jpg.rf.5cf01c958251351680a06b3a9fced35a.jpg
Missing label for: Aerial_Location_1_103_jpg.rf.535d12004686021a8d647c36cffcefe1.jpg
Missing label for: Aerial_Location_1_104_jpg.rf.ee028aea28f3bbe7aa16838224499521.jpg
Missing label for: Aerial_Location_1_105_jpg.rf.3cc6804b02bedaa43af6cb60deaa8aa2.jpg
Missing label for: Aerial_Location_1_106_jpg.rf.4cc8646926784d6796589d6c1a157d88.jpg
Missing label for: Aerial_Location_1_109_jpg.rf.56e13dba421ae282208464b13933dc87.jpg
Missing label for: Aerial_Location_1_109_jpg.rf.a71706f14d9f3d9d5b70cd2b68e281ab.jpg
Missing label for: Aerial_Location_1_110_jpg.rf.8d015e23dbe6e8bcd4446017272e9318.jpg
Mis

In [31]:
find /Users/surisettivamsikrishna/Downloads/Vamsi/Pc/CODES/Mark2/River/Ganges/Dataset/ -name".DS_Store" -delete


SyntaxError: invalid syntax (2492401162.py, line 1)

In [33]:
import os
import shutil

train_images = "/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset/train/images"
train_labels = "/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset/train/labels"

# Ensure matching image and label files
image_files = {os.path.splitext(img)[0] for img in os.listdir(train_images)}
label_files = {os.path.splitext(lbl)[0] for lbl in os.listdir(train_labels)}

# Find unmatched files
unmatched_images = image_files - label_files
unmatched_labels = label_files - image_files

# Remove unmatched files
for img in unmatched_images:
    os.remove(os.path.join(train_images, f"{img}.jpg"))

for lbl in unmatched_labels:
    os.remove(os.path.join(train_labels, f"{lbl}.txt"))

print("Unmatched files removed!")


FileNotFoundError: [Errno 2] No such file or directory: '/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/River Ganges/Dataset/train/images/.DS_Store.jpg'